En este código se importa la base de datos "Datos_Climatologicos_Unificados.xlsx", que contiene el valor real de precipitación diaria para cada estación desde 01-ene-1980 hasta 09-nov-2024. También se importa la base de datos "PronósticosNextGen.xlsx", que contiene el pronóstico del modelo NextGen para la precipitación acumulada por período estacional en cada una de las diferentes estaciones meteorológicas. Finalmente se importa "Climatologia_por_Periodos_Estacionales.xlsx", que contiene la media y los percentiles 1/3 y 2/3 para el conjunto de precipitaciones acumuladas de la Climatología (1991-2020) para cada período estacional en las diferentes estaciones meteorológicas.

Para cada estación meteorológica y período estacional en la base de datos "PronósticosNextGen.xlsx", se va a calcular el valor de precipitación acumulada real correspondiente en "Datos_Climatologicos_Unificados.xlsx", y también se busca la media y percentiles en la base de datos "Climatologia_por_Periodos_Estacionales.xlsx", con lo que se puede definir el valor para la variable objetivo:

- Clasificación de la precipitación del día siguiente en cotas percentiles

El valor de esta variable se inserta como una nueva columna en el dataframe "PronósticosNextGen.xlsx", el cual se exporta al final con el nombre "DataNextGen.xlsx"

# IMPORTACIÓN DE DATOS Y LIBRERÍAS

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Cargar datos desde Excel

climatologia_df = pd.read_excel('Climatologia_por_Periodos_Estacionales.xlsx')  # Archivo con climatología (media y percentiles de PrecAcum para cada estación meteorológica en los diferentes períodos estacionales)
Datos_Reales = pd.read_excel('Datos_Climatologicos_Unificados.xlsx') # Datos de precipitación diaria reales
PronosticosNextGen = pd.read_excel('PronósticosNextGen.xlsx') # Contiene la variable objetivo predicha por NextGen

In [ ]:
print(climatologia_df)
print(Datos_Reales)
print(PronosticosNextGen)

# DEFINICIÓN DE FUNCIONES

In [ ]:
# Calcular el acumulado de precipitación para una estación meteorológica en un período estacional definido

def Calcular_prec_acum(estacion, Periodo, Año):
    
    # Creamos un filtro para la estación seleccionada
    filtro_estacion = Datos_Reales['ESTACIÓN'] == estacion
    
    # Creamos el rango de fechas dependiendo del período seleccionado
    if Periodo == 'MJJ':
        # Mayo, Junio, Julio del año X
        fecha_inicio = f'{Año}-05-01'
        fecha_fin = f'{Año}-07-31'
    elif Periodo == 'ASO':
        # Agosto, Septiembre, Octubre del año X
        fecha_inicio = f'{Año}-08-01'
        fecha_fin = f'{Año}-10-31'
    elif Periodo == 'DEFM':
        # Diciembre del año X-1 y Enero, Febrero, Marzo del año X
        fecha_inicio = f'{Año-1}-12-01'
        fecha_fin = f'{Año}-03-31'
        
    # Creamos el filtro de fechas
    filtro_fecha = (Datos_Reales['FECHA'] >= fecha_inicio) & (Datos_Reales['FECHA'] <= fecha_fin)
    
    # Filtramos los datos
    datos_filtrados = Datos_Reales[filtro_estacion & filtro_fecha]
    
    # Calculamos el acumulado de lluvia en el período especificado
    acumulado_lluvia = datos_filtrados['PRECIPITACIÓN'].sum()
    
    return acumulado_lluvia

In [ ]:
# Función para clasificar por percentiles

def clasificar_percentiles(prec_Acum, p1_3, p2_3):
    if p1_3 == 0 and p2_3 == 0:
        return 0
    else:
        if prec_Acum <= p1_3:
            return -1
        elif prec_Acum >= p2_3:
            return 1
        else:
            return 0

# PROCESAMIENTO DE DATOS

In [ ]:
DataNextGen = PronosticosNextGen.copy()

# Inicializar la nueva columna
DataNextGen['percentil_Real'] = None

# Iterar sobre las filas de los pronósticos
for i, row in DataNextGen.iterrows():
    
    # Obtener la estación y la fecha de la fila actual
    estacion = row['Estacion']
    Periodo = row['Periodo']
    Año = row['Año']
    
    # Buscar el valor real de precipitación acumulada en Datos_Reales para esa estación, año, y período
    precipAcum_real = Calcular_prec_acum(estacion, Periodo, Año)
    
    # Buscar la media y los percentiles en climatologia_df para esa estación y fecha
    climatologia = climatologia_df[(climatologia_df['ESTACIÓN'] == estacion) & (climatologia_df['Periodo_estacional'] == Periodo)]
    
    # Obtener la media y percentiles del dataframe climatologia_df
    media = climatologia['Media'].values[0]
    p1_3 = climatologia['Percentil_1_3'].values[0]
    p2_3 = climatologia['Percentil_2_3'].values[0]
    
    # Ejecutar la función para calcular la variable objetivo real
    clasificacion_percentiles = clasificar_percentiles(precipAcum_real, p1_3, p2_3)
    
    # Almacenar los resultados en la nueva columna de DataNextGen
    DataNextGen.at[i, 'percentil_Real'] = clasificacion_percentiles

In [ ]:
# Exportar a Excel
DataNextGen.to_excel('DataNextGen.xlsx', index=False)